In [1]:
import pickle
import numpy as np
import numpy.ma as ma

# ДЗ 10 от 19.10.20

## Задание

Реализовать языковую модель информационного поиска для поиска предложений в Википедии для $\lambda_1 = 0.5$  и $\lambda_2 = 0.9$



## Загружаем лемматизированные тексты из предыдущего задания

In [2]:
queries = [
    'Пейзаж с выжженной равниной увековечил образ вечного города.',
    'Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.',
    'Трезубец на жёлто-синем фоне — это флаг Барбадоса.'
]

with open('../Task_4/lemmatized.pkl', 'rb') as output_file:
    docs, original_sentences, queries_processed = pickle.load(output_file)

Убираем документы без токенов

In [3]:

docs = [[t for t in tokens if len(t)>1] for tokens in docs]

original_sentences = [sent for tokens, sent in zip(docs, original_sentences) if len(tokens)>0]
docs = [tokens for tokens in docs if len(tokens)>0]

Векторизуем документы и запросы

In [18]:
def vectorize(docs, vocab=None):
    '''
    docs - list of lists of tokens
    vocab - set of tokens
    Returns count array with shape (N x V), where N - number of documents and V - size of vocab
    '''
    if vocab is None:
        vocab = set([token for sentence in docs for token in sentence if len(token)>0])
    vectorized = np.zeros((len(docs),len(vocab)))

    token_ind_dict = {tok: ind for ind, tok in dict(enumerate(vocab)).items()}

    for doc_ind, doc in enumerate(docs):
        for token in doc:
            try:
                token_id = token_ind_dict[token]
                vectorized[doc_ind][token_id] += 1
            except KeyError:
                pass
    return vectorized, token_ind_dict

In [44]:
vocab = set([token for sentence in docs for token in sentence if len(token)>1])
doc_vec, token_ind_dict = vectorize(docs, vocab)
queries_vec, _ = vectorize(queries_processed, vocab)

In [ ]:
def get_lm_probs(query_vec, doc_vecs, lmbd):

    query_terms = np.where(query_vec)[0]
    corpus_freqs = doc_vec.sum(axis=0)
    corpus_size = corpus_freqs.sum()
    doc_sizes = doc_vec.sum(axis=1)

    global_probs = corpus_freqs[query_terms] / corpus_size
    local_probs = (doc_vec[:,query_terms].T / doc_sizes).T

    doc_probs = np.prod(global_probs * lmbd + local_probs * (1-lmbd), axis=1)

    return doc_probs
query_results = get_lm_probs(queries_vec[0], doc_vecs, 0.5).argsort()[-3:]

In [81]:
query_terms = np.where(queries_vec[0])[0]
corpus_freqs = doc_vec.sum(axis=0)
corpus_size = corpus_freqs.sum()
doc_sizes = doc_vec.sum(axis=1)

global_probs = corpus_freqs[query_terms] / corpus_size
local_probs = (doc_vec[:,query_terms].T / doc_sizes).T

lmbd = 0.9
doc_probs = np.prod(global_probs * lmbd + local_probs * (1-lmbd), axis=1)

In [95]:
np.prod((doc_vec[:,query_terms].T / doc_sizes).T + global_probs, axis=1).argsort()[-3:]

array([82, 92, 87], dtype=int64)

In [98]:
original_sentences[82]

'roma ˈroːma  столица и крупнейший город италии'

In [75]:
lmbd = 0.9
np.prod(global_probs * 0.9 + local_probs * 0.1, axis=1).argsort()[-5:]

array([36, 47, 63, 68, 44], dtype=int64)

In [62]:
# return doc_probs
query_results = doc_probs.argsort()[-3:]
query_results

array([82, 92, 87], dtype=int64)

In [80]:
original_sentences[36]

'биологическое оружие включает также средства доставки патогенных микроорганизмов и животныхпереносчиков'

array([82, 92, 87], dtype=int64)

In [63]:
original_sentences[45]

'оружейная палата  лучшие компьютерные игры'

In [21]:
ind_token_dict = {v:k for k, v in token_ind_dict.items()}

In [23]:
ind_token_dict[565]

'вечный'

In [15]:
queries_processed[0]

['пейзаж',
 'с',
 'выжигать',
 'равнина',
 'увековечивать',
 'образ',
 'вечный',
 'город']

In [13]:
local_probs

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.

In [132]:
query_results = get_lm_probs(queries_vec[0], doc_vec, lmbd=0.5).argsort()[-3:]

array([82, 92, 87], dtype=int64)

In [139]:
query_results = get_lm_probs(queries_vec[0], doc_vec, lmbd=0.5).argsort()[-3:]

'также была осуществлена реформа судебной системы штата для финансирования необходимых программ был увеличен налог на имущество'

In [138]:
query_results = get_lm_probs(queries_vec[0], doc_vec, lmbd=0.5).argsort()[-3:]

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight  | Sentence
(82  0.000017)  также была осуществлена реформа судебной системы штата для финансирования необходимых программ был увеличен налог на имущество
(92  0.000017)  
(87  0.000078)  blackburn luke pryor  the kentucky encyclopedia неопр
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight  | Sentence
(63  0.000000)  в апреле 1865 года к консулу сша в торонто явился некий джозеф хьямс и рассказал ему о заговоре во главе которого стоит находящийся в канаде блэкберн
(68  0.000000)  под рубашками лежало одеяло по утверждению хьямса взятое с постели жертвы жёлтой лихорадки
(44  0.000000)  флаг барбадоса  официальный символ государства барбадос
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight  | Sentence
(84  0.000000)  блэкберн был дважды женат у него был один сын
(8  0.000000)  
(34  0.000000)  зел

In [128]:
original_sentences[82]

'roma ˈroːma  столица и крупнейший город италии'